🔹 Etapa 1 – Imports

In [1]:
import pandas as pd
from data_clean import DataClean
from transform import tempo_voo_horas_para_minutos, classifica_turno_partida
from utils import create_db
from sqlalchemy import create_engine


🔹 Etapa 2 – Carregar dados

In [2]:
url = "https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv"
df = pd.read_csv(url)
df.head()


,Unnamed: 0,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,1,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0
1,2,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0
2,3,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0
3,4,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0
4,5,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0


🔹 Etapa 3 – Criar config de colunas

In [10]:
config_dict = {
    "id": [0, 1, 2, 3, 4, 5, 6, 7],
    "nome_original": ["dep_time", "arr_delay", "carrier", "flight", "air_time", "distance", "origin", "dest"],
    "nome": ["data_hora", "atraso_chegada", "companhia", "id_voo", "tempo_voo", "distancia", "origem", "destino"],
    "aceita_negativo": [0, 1, 1, 1, 1, 1, 1, 1],
    "tipo": ["date", "float", "str", "str", "float", "float", "str", "str"]
}
config_df = pd.DataFrame(config_dict)

print(df.columns.tolist())

['Unnamed: 0', 'year', 'month', 'day', 'dep_time', 'dep_delay', 'arr_time', 'arr_delay', 'carrier', 'tailnum', 'flight', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute']


🔹 Etapa 4 – Aplicar limpeza com DataClean

In [12]:
limpador = DataClean(df, config_df)
limpador.select_cols()
limpador.rename_cols()
limpador.select_nnull_cols()
limpador.select_nneg_cols()

df_limpo = limpador.return_data()
df_limpo.head()


,data_hora,atraso_chegada,companhia,id_voo,tempo_voo,distancia,origem,dest
0,517.0,11.0,UA,1545,227.0,1400,EWR,IAH
1,533.0,20.0,UA,1714,227.0,1416,LGA,IAH
2,542.0,33.0,AA,1141,160.0,1089,JFK,MIA
3,544.0,-18.0,B6,725,183.0,1576,JFK,BQN
4,554.0,-25.0,DL,461,116.0,762,LGA,ATL


🔹 Etapa 5 – Aplicar transformações

In [13]:
df_transformado = tempo_voo_horas_para_minutos(df_limpo)
df_transformado = classifica_turno_partida(df_transformado)

df_transformado[['tempo_voo', 'tempo_voo_minutos', 'data_hora', 'turno_partida']].head()


,tempo_voo,tempo_voo_minutos,data_hora,turno_partida
0,227.0,13620.0,1970-01-01 00:00:00.000000517,MADRUGADA
1,227.0,13620.0,1970-01-01 00:00:00.000000533,MADRUGADA
2,160.0,9600.0,1970-01-01 00:00:00.000000542,MADRUGADA
3,183.0,10980.0,1970-01-01 00:00:00.000000544,MADRUGADA
4,116.0,6960.0,1970-01-01 00:00:00.000000554,MADRUGADA


🔹 Etapa 6 – Criar banco e salvar dados

In [17]:

engine = create_engine('sqlite:///voos01.db')
create_db(engine)
df_transformado.to_sql('flights', con=engine, if_exists='replace', index=False)

print("Dados transformados e salvos com sucesso.")


Dados transformados e salvos com sucesso.
